In [1]:
import os
import csv
import time
import socket
import logging
import statistics
import requests
import pandas as pd
from datetime import datetime
from urllib.parse import urlparse

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [3]:
class SpeedTester:
    def __init__(self, num_requests=3):
        self.num_requests = num_requests
        self.unreachable_sites = []
        self.results_file = 'speed_test_results.csv'
        self._initialize_results_file()

    def _initialize_results_file(self):
        if not os.path.exists(self.results_file):
            with open(self.results_file, 'w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=['url', 'timestamp', 'dns_lookup', 'connection', 'ttfb', 'download', 'total', 'status', 'error_message'])
                writer.writeheader()

    def _read_tested_sites(self):
        if os.path.exists(self.results_file):
            with open(self.results_file, 'r') as f:
                reader = csv.DictReader(f)
                return {row['url']: row['status'] for row in reader}
        return {}

    def is_website_reachable(self, url):
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

        try:
            response = requests.head(url, headers=headers, timeout=10, allow_redirects=True)
            return 200 <= response.status_code < 400
        except requests.RequestException as e:
            logging.warning(f"Website {url} unreachable: {e}")
            return False

    def _measure_dns(self, domain):
        start_time = time.time()
        try:
            socket.gethostbyname(domain)
            dns_time = time.time() - start_time
            return dns_time, True, ''
        except Exception as e:
            return None, False, str(e)

    def _test_single_url(self, url):
        logging.info(f"Starting test for {url}")

        metrics = {
            'url': url,
            'timestamp': datetime.now().isoformat(),
            'dns_lookup': [],
            'connection': [],
            'ttfb': [],
            'download': [],
            'total': [],
            'status': 'success',
            'error_message': ''
        }

        if not self.is_website_reachable(url):
            metrics['status'] = 'failed'
            metrics['error_message'] = 'Website unreachable'
            self.unreachable_sites.append({'url': url, 'error': 'Website unreachable'})
            return metrics

        parsed_url = urlparse(url)
        domain = parsed_url.netloc

        dns_time, dns_success, dns_error = self._measure_dns(domain)
        if not dns_success:
            metrics['status'] = 'failed'
            metrics['error_message'] = dns_error
            self.unreachable_sites.append({'url': url, 'error': dns_error})
            return metrics

        logging.info(f"Starting {self.num_requests} requests for {url}")
        for i in range(self.num_requests):
            logging.info(f"Request {i + 1}/{self.num_requests} for {url}")
            try:
                headers = {
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
                }

                metrics['dns_lookup'].append(dns_time)
                start_time = time.time()

                with requests.Session() as session:
                    conn_start = time.time()
                    response = session.get(url, headers=headers, stream=True, timeout=10, verify=False, allow_redirects=True)
                    conn_time = time.time() - conn_start

                    if not (200 <= response.status_code < 400):
                        raise requests.RequestException(f"Invalid status code: {response.status_code}")

                    end_time = time.time()
                    total_time = end_time - start_time
                    download_time = total_time - conn_time

                    metrics['connection'].append(conn_time)
                    metrics['download'].append(download_time)
                    metrics['total'].append(total_time)
                    metrics['ttfb'].append(response.elapsed.total_seconds())

                time.sleep(1)

            except Exception as e:
                logging.error(f"Error during request {i + 1} for {url}: {str(e)}")
                if i == 0:
                    metrics['status'] = 'failed'
                    metrics['error_message'] = str(e)
                    self.unreachable_sites.append({'url': url, 'error': str(e)})
                    break

        for key in ['dns_lookup', 'connection', 'ttfb', 'download', 'total']:
            if metrics[key]:
                metrics[key] = statistics.mean(metrics[key])
            else:
                metrics[key] = None

        logging.info(f"Completed testing {url}")

        with open(self.results_file, 'a', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'timestamp', 'dns_lookup', 'connection', 'ttfb', 'download', 'total', 'status', 'error_message'])
            writer.writerow(metrics)

        return metrics

    def test_urls(self, urls):
        results = []
        tested_sites = self._read_tested_sites()
        
        for url in urls:
            if tested_sites.get(url) == 'success':
                logging.info(f"Skipping {url}, already tested successfully.")
                continue
            result = self._test_single_url(url)
            results.append(result)
        
        return pd.DataFrame(results)

In [ ]:
def main():
    try:
        logging.info("Starting speed test script")
        
        # Load websites from CSV and format URLs
        websites = pd.read_csv("../data/us_gov_domain_list.csv", usecols=["Domain name"]).drop_duplicates()
        websites = websites["Domain name"].dropna().astype(str).tolist()
        
        # Add https:// to URLs that don't have a scheme
        websites = ['https://' + site if not site.startswith(('http://', 'https://')) else site 
                   for site in websites]
        
        # Initialize tester and run tests
        tester = SpeedTester(num_requests=3)
        results_df = tester.test_urls(websites)
        
        # Print summary
        print("\nResults Summary:")
        print(f"Total sites tested: {len(websites)}")
        print(f"Successful: {len(results_df[results_df['status'] == 'success'])}")
        print(f"Failed: {len(results_df[results_df['status'] == 'failed'])}")
        
        # Print detailed results
        print("\nDetailed Results (in seconds):")
        display_columns = ['url', 'status', 'error_message'] + [
            col for col in results_df.columns 
            if col not in ['url', 'status', 'error_message', 'timestamp']
        ]
        print(results_df[display_columns].round(3))
        
    except Exception as e:
        logging.error(f"Script error: {str(e)}")
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()

2025-01-26 19:40:17,962 - INFO - Starting speed test script
2025-01-26 19:40:17,996 - INFO - Starting test for https://36thdistrictcourtmi.gov
2025-01-26 19:40:18,914 - INFO - Starting 3 requests for https://36thdistrictcourtmi.gov
2025-01-26 19:40:18,916 - INFO - Request 1/3 for https://36thdistrictcourtmi.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '36thdistrictcourtmi.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.36thdistrictcourt.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2025-01-26 19

2025-01-26 19:40:40,546 - INFO - Request 3/3 for https://abingdon-va.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'abingdon-va.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2025-01-26 19:40:41,828 - INFO - Completed testing https://abingdon-va.gov
2025-01-26 19:40:41,832 - INFO - Starting test for https://abingtonma.gov
2025-01-26 19:40:42,466 - INFO - Starting 3 requests for https://abingtonma.gov
2025-01-26 19:40:42,477 - INFO - Request 1/3 for https://abingtonma.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'abingtonma.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#s

2025-01-26 19:41:04,386 - INFO - Starting 3 requests for https://acworth-ga.gov
2025-01-26 19:41:04,388 - INFO - Request 1/3 for https://acworth-ga.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'acworth-ga.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2025-01-26 19:41:08,167 - INFO - Request 2/3 for https://acworth-ga.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'acworth-ga.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2025-01-26 19:41:09,569 - INFO - Request 3/3 for https://acworth-ga.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-pa

2025-01-26 19:41:36,320 - INFO - Request 2/3 for https://addisontownshipmi.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'addisontownshipmi.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2025-01-26 19:41:38,844 - INFO - Request 3/3 for https://addisontownshipmi.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'addisontownshipmi.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2025-01-26 19:41:41,290 - INFO - Completed testing https://addisontownshipmi.gov
2025-01-26 19:41:41,299 - INFO - Starting test for https://addisontx.gov
2025-01-26 19:41:43,

2025-01-26 19:42:02,595 - INFO - Completed testing https://adelantoca.gov
2025-01-26 19:42:02,600 - INFO - Starting test for https://adeliowa.gov
2025-01-26 19:42:02,933 - INFO - Starting 3 requests for https://adeliowa.gov
2025-01-26 19:42:02,935 - INFO - Request 1/3 for https://adeliowa.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adeliowa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.adeliowa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2025-01-26 19:42:04,244 - INFO - Request 2/3 for htt

...

2025-01-26 19:46:32,662 - INFO - Request 2/3 for https://altusok.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'altusok.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2025-01-26 19:46:34,197 - INFO - Request 3/3 for https://altusok.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'altusok.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2025-01-26 19:46:35,711 - INFO - Completed testing https://altusok.gov
2025-01-26 19:46:35,719 - INFO - Starting test for https://alvin.gov
2025-01-26 19:46:36,586 - INFO - Starting 3 requests for https://alvin.gov

2025-01-26 19:47:03,399 - INFO - Completed testing https://alvordtx.gov
2025-01-26 19:47:03,409 - INFO - Starting test for https://amadorcity.gov
2025-01-26 19:47:05,352 - INFO - Starting 3 requests for https://amadorcity.gov
2025-01-26 19:47:05,354 - INFO - Request 1/3 for https://amadorcity.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'amadorcity.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'amador-city.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2025-01-26 19:47:07,740 - INFO - Request 2/3 fo

2025-01-26 19:47:41,268 - INFO - Completed testing https://americanfork.gov
2025-01-26 19:47:41,276 - INFO - Starting test for https://americusga.gov
2025-01-26 19:47:41,533 - INFO - Starting 3 requests for https://americusga.gov
2025-01-26 19:47:41,534 - INFO - Request 1/3 for https://americusga.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'americusga.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2025-01-26 19:47:42,675 - INFO - Request 2/3 for https://americusga.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'americusga.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ss

2025-01-26 19:48:10,064 - INFO - Request 3/3 for https://amherstnh.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'amherstnh.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.amherstnh.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2025-01-26 19:48:11,978 - INFO - Completed testing https://amherstnh.gov
2025-01-26 19:48:11,986 - INFO - Starting test for https://amherstva.gov
2025-01-26 19:48:12,131 - INFO - Starting 3 requests for https://amherstva.gov
2025-01-26 19:48:12,133 - INFO - Request 1/3 for

2025-01-26 19:48:36,348 - INFO - Request 3/3 for https://anacorteswa.gov
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'anacorteswa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.anacorteswa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2025-01-26 19:48:38,227 - INFO - Completed testing https://anacorteswa.gov
2025-01-26 19:48:38,233 - INFO - Starting test for https://anaheim.gov
2025-01-26 19:48:38,242 - WARNING - Website https://anaheim.gov unreachable: HTTPSConnectionPool(host='anaheim.gov', port

In [5]:
results_df.head()

NameError: name 'results_df' is not defined